# Analytics

#### Date: 2020/02

#### SUMMARY:

- This notebook represents the project quality analysis of the date exposed right above. 

### TEAM:

##### Semester: 2020/02
##### Professor: Hilmer Neri

##### Members:

- Member x
- Member y

### LIBRARIES

In [1]:
# Deal with data
import pandas as pd
import numpy as np
import json

# Deal with API request
import urllib3
from urllib3 import request

# Deal with visualization
import seaborn as sns
import matplotlib.pyplot as plt

### GRAPH SETTINGS

In [2]:
%config InlineBackend.figure_format ='retina'
sns.set(font_scale=1.5)
sns.set_style('darkgrid',
              {'xtick.bottom' : True,
               'ytick.left': True,
               'grid.linestyle':'--',
               'font.monospace': ['Computer Modern Typewriter'],
               'axes.edgecolor' : 'white'})

### DATAFRAME SETTINGS

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

### SonarCloud

#### KEYS

In [4]:
front_key = 'fga-eps-mds_2020.1-VC_Gestor-FrontEnd'

In [5]:
back_key = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd'

#### METRICS

In [6]:
metric_list = ['files',
               'functions',
               'complexity',
               'comment_lines_density',
               'duplicated_lines_density',
               'security_rating',
               'tests',
               'test_success_density',
               'test_execution_time',
               'reliability_rating']

len(metric_list)

10

In [7]:
def generate_metric_string(metric_list):
    metric_str = ''
    
    for metric in metric_list:
        metric_str += metric + ','
        
    return metric_str

In [8]:
metric_str = generate_metric_string(metric_list)

#### URLS

In [9]:
front_url = f"https://sonarcloud.io/api/measures/component_tree?component={front_key}&metricKeys={metric_str}&ps=500"
back_url = f"https://sonarcloud.io/api/measures/component_tree?component={back_key}&metricKeys={metric_str}&ps=500"

#### API REQUEST

In [10]:
http = urllib3.PoolManager()

In [11]:
front_request = http.request('GET', front_url)
front_request.status

200

In [12]:
back_request = http.request('GET', back_url)
back_request.status

200

#### JSON DECODING

In [13]:
front_json = json.loads(front_request.data.decode('utf-8'))
back_json = json.loads(back_request.data.decode('utf-8'))

## DATA

### PROJECT

In [14]:
project_front_json = front_json['baseComponent']['measures']
project_back_json = back_json['baseComponent']['measures']

In [15]:
project_front_data = pd.DataFrame(project_front_json)
project_back_data = pd.DataFrame(project_back_json)

##### FRONT

In [16]:
project_front_data

,metric,value,bestValue
0,duplicated_lines_density,4.3,False
1,functions,263,NaN
2,security_rating,1.0,True
3,files,45,NaN
4,complexity,412,NaN
5,reliability_rating,4.0,False
6,comment_lines_density,4.7,False


##### BACK

In [17]:
project_back_data

,metric,value,bestValue
0,duplicated_lines_density,48.0,False
1,functions,91,NaN
2,security_rating,1.0,True
3,files,80,NaN
4,complexity,173,NaN
5,reliability_rating,3.0,False
6,comment_lines_density,1.0,False


### FILES

In [18]:
def metric_per_file(json):
    file_json = []
    
    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)
            
    return file_json

In [19]:
front_file_json = metric_per_file(front_json)
back_file_json = metric_per_file(back_json)

In [20]:
def generate_file_dataframe(metric_list, json, language_extension):
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)
    
    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass
        
    df.reset_index(inplace = True)
    df = df.rename({'index': 'path'}, axis=1).drop(['files'], axis=1)

    return df

In [21]:
front_files_data = generate_file_dataframe(metric_list, front_file_json, language_extension = 'js')
back_files_data = generate_file_dataframe(metric_list, back_file_json, language_extension = 'js')

##### FRONT

In [22]:
front_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,src/components/404/404.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
1,src/components/pages/Anuncio.js,5,7,9.8,10.9,1.0,NaN,100.0,NaN,1.0
2,src/services/apiBeneficio.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
3,src/services/apiNoticias.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
4,src/services/apiPostagem.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
5,src/services/apiUser.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
6,src/components/app/App.test.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
7,src/components/pages/BeneficiosCriar.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
8,src/components/pages/BeneficiosEditar.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
9,src/components/pages/BeneficiosGerenciar.js,8,10,5.4,0.0,1.0,NaN,100.0,NaN,1.0


##### BACK

In [23]:
back_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,packages/user/src/controller/AuthenticateUserS...,1,1,8.3,0.0,1.0,NaN,100.0,NaN,1.0
1,packages/user/src/controller/AuthResolve.js,1,3,4.3,0.0,1.0,NaN,100.0,NaN,1.0
2,packages/benefits/src/models/benefit.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
3,packages/benefits/src/controller/BenefitContro...,5,5,29.0,0.0,1.0,NaN,100.0,NaN,1.0
4,packages/benefits/src/test/BenefitsController....,12,12,0.0,0.0,1.0,NaN,100.0,NaN,1.0
5,packages/user/src/controller/ChangePasswordRes...,1,5,0.0,0.0,1.0,NaN,100.0,NaN,1.0
6,packages/user/src/controller/CheckCodeResolve.js,1,4,0.0,0.0,1.0,NaN,100.0,NaN,1.0
7,packages/news/src/test/ControllerTesting.spec.js,13,13,0.0,0.0,1.0,NaN,100.0,NaN,1.0
8,packages/user/src/test/ControllerTesting.spec.js,24,24,0.0,33.4,1.0,NaN,100.0,NaN,1.0
9,packages/benefits/src/controller/CreateBenefit...,1,4,0.0,0.0,1.0,NaN,100.0,NaN,1.0


# ANALYSIS

## MAINTAINABILITY

### CODE QUALITY

##### COMPLEXITY

In [24]:
def m1(df):
    
    density_non_complex_files = round((len(df[(df['complexity'].astype(float)/df['functions'].astype(float)) < 10])/len(df))*100, 2)
    
    return density_non_complex_files

##### COMMENTS

In [25]:
def m2(df):
    
    density_comment_files = round((len(df[(df['comment_lines_density'].astype(float) > 10) & (df['comment_lines_density'].astype(float) < 30)])/len(df))*100, 2)
    
    return density_comment_files

##### DUPLICATIONS

In [26]:
def m3(df):
    
    duplication = round((len(df[(df['duplicated_lines_density'].astype(float) < 5)])/len(df))*100, 2)
    
    return duplication

### BLOCKING CODE

#### NON-BLOCKING FILES

In [27]:
def m4(df):
    
    non_blocking_files = round((len(df[(df['security_rating'].astype(float) >= 4)])/len(df))*100,2)
    
    return non_blocking_files

## RELIABILITY

#### TEST SUCCESS

In [28]:
def m5(df):
    
    test_success_file = df[['path', 'test_success_density']]
    test_success_repository = df['test_success_density'].astype(float).mean()
    
    print("Project test unit density: ", test_success_repository)
    
    return test_success_file

#### FAST TESTS

In [29]:
def m6(df):
    
    fast_test_df = df[(df['test_execution_time'].astype(float) < 300)]
    fast_test_df['fast_test'] = fast_test_df['test_execution_time']/fast_test_df['tests']
    
    fast_test_file = fast_test_df[['path', 'fast_test']]
    
    fast_test_repository = fast_test_df['fast_test'].astype(float).mean()
    
    print("Project test unit density: ", fast_test_repository)
    
    return fast_test_file

## PRODUCTIVITY

### TIMEFRAME: PUT_YOUR_TIMEFRAME_HERE

In [30]:
NUMBER_OF_ISSUES_RESOLVED = 7
NUMBER_OF_ISSUES = 50

TAGS = {
    "HOTFIX": 4,
    "DOCS": 2,
    "FEATURE": 3,
    "ARQ": 2,
    "DEVOPS": 5,
    "ANALYTICS": 2,
    "US": 4,
    "EASY": 9,
    "MEDIUM": 1,
    "HARD": 2,
    "EPS": 5,
    "MDS": 7
}

#### RESOLVED ISSUES' THROUGHPUT

In [31]:
def m7(number_of_issues_resolved, number_of_issues):
    
    resolved_issues_throughput = round((number_of_issues_resolved / number_of_issues) * 100, 2)
    
    return resolved_issues_throughput

#### ISSUE TYPE IN A TIMEFRAME

In [32]:
def density(issue, number_of_issues):
    issue_density = round((issue / number_of_issues) * 100, 2)
    return issue_density

In [33]:
def m8(tag_dict, number_of_issues):
    
    issue_densities = {
        "hotfix": [density(tag_dict["HOTFIX"], number_of_issues)],
        "docs": [density(tag_dict["DOCS"], number_of_issues)],
        "feature": [density(tag_dict["FEATURE"], number_of_issues)],
        "arq": [density(tag_dict["ARQ"], number_of_issues)],
        "devops": [density(tag_dict["DEVOPS"], number_of_issues)],
        "analytics": [density(tag_dict["ANALYTICS"], number_of_issues)],
        "us": [density(tag_dict["US"], number_of_issues)],
        "easy": [density(tag_dict["EASY"], number_of_issues)],
        "medium": [density(tag_dict["MEDIUM"], number_of_issues)],
        "hard": [density(tag_dict["HARD"], number_of_issues)],
        "eps": [density(tag_dict["EPS"], number_of_issues)],
        "mds": [density(tag_dict["MDS"], number_of_issues)]
    }

    issue_densities = pd.DataFrame.from_dict(issue_densities).T.reset_index()
    
    issue_densities.columns = ['density' ,'percentage']
    
    return issue_densities

#### BUGS RATIO

In [34]:
def m9(tag_dict, number_of_issues):
    
    bugs_ratio = round(((tag_dict["DOCS"] + tag_dict["FEATURE"] + tag_dict["ARQ"] + tag_dict["DEVOPS"] + tag_dict["ANALYTICS"]) / number_of_issues) * 100, 2)
    
    return bugs_ratio

### METRIC RESULTS

In [35]:
m1(back_files_data)

83.87

In [36]:
m2(back_files_data)

9.68

In [37]:
m3(back_files_data)

90.32

In [38]:
m4(back_files_data)

0.0

In [39]:
m5(back_files_data)

Project test unit density:  100.0


,path,test_success_density
0,packages/user/src/controller/AuthenticateUserS...,100.0
1,packages/user/src/controller/AuthResolve.js,100.0
2,packages/benefits/src/models/benefit.js,100.0
3,packages/benefits/src/controller/BenefitContro...,100.0
4,packages/benefits/src/test/BenefitsController....,100.0
5,packages/user/src/controller/ChangePasswordRes...,100.0
6,packages/user/src/controller/CheckCodeResolve.js,100.0
7,packages/news/src/test/ControllerTesting.spec.js,100.0
8,packages/user/src/test/ControllerTesting.spec.js,100.0
9,packages/benefits/src/controller/CreateBenefit...,100.0


In [40]:
m6(back_files_data)

Project test unit density:  nan


,path,fast_test


In [41]:
m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

14.0

In [42]:
m8(TAGS, NUMBER_OF_ISSUES)

,density,percentage
0,hotfix,8.0
1,docs,4.0
2,feature,6.0
3,arq,4.0
4,devops,10.0
5,analytics,4.0
6,us,8.0
7,easy,18.0
8,medium,2.0
9,hard,4.0


In [43]:
m9(TAGS, NUMBER_OF_ISSUES)

28.0